
BERT en un problema de clasificación
====================================

Introducción
------------

Los modelos basados en transformers nos pueden ayudar a resolver varios tipos de problemas. Desde problemas de clasificación y regresión hasta tareas más complejas como resumen de textos o generación de leguaje condicionado. Veamos como resolver el problema de clasificación de tweets sobre el que hemos estado trabajando anteriormente pero ahora utilizando el modelo BERT.

### Para ejecutar este notebook

Para ejecutar este notebook, instale las siguientes librerias:

In [1]:
!wget https://raw.githubusercontent.com/santiagxf/M72109/master/NLP/Datasets/mascorpus/tweets_marketing.csv \
    --quiet --no-clobber --directory-prefix ./Datasets/mascorpus/

!wget https://raw.githubusercontent.com/santiagxf/M72109/master/NLP/Utils/TextDataset.py \
    --quiet --no-clobber --directory-prefix ./Utils/
    
!wget https://raw.githubusercontent.com/santiagxf/M72109/master/docs/nlp/neural/BERT.txt \
    --quiet --no-clobber
!pip install -r BERT.txt --quiet

     |████████████████████████████████| 3.1 MB 4.3 MB/s 
     |████████████████████████████████| 895 kB 33.8 MB/s 
     |████████████████████████████████| 59 kB 6.3 MB/s 
     |████████████████████████████████| 596 kB 45.2 MB/s 
     |████████████████████████████████| 3.3 MB 37.9 MB/s 


In [2]:
import warnings
warnings.filterwarnings('ignore')

Cargamos el set de datos

In [3]:
import pandas as pd

tweets = pd.read_csv('Datasets/mascorpus/tweets_marketing.csv')

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(tweets['TEXTO'], tweets['SECTOR'], 
                                                    test_size=0.33, 
                                                    stratify=tweets['SECTOR'])

### Verificando el hardware disponible

In [5]:
import torch
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

print("Este notebook se está ejecutando en", device)

Este notebook se está ejecutando en cuda


Transferencia de Aprendizaje y Fine-tuning
------------------------------------------

En general, existen 2 estrategias para utilizar modelos de lenguaje pre-entrenados en una tarea especifica:
 - Feature-based
 - Fine-tunning
 
Las técnicas que se conocen como **Feature-based** utiliza arquitecturas especificas para resolver cada una de las tareas de NLP, en donde los pesos de las representaciones vectoriales están "congeladas" y no son parámetros que el modelo deba optimizar. En consecuencia, estos modelos son más rápidos de entrenar y permiten aplicar arquitecturas especificas que sean diferenciales en cada una de las tareas.
 
Por el otro lado, las técnicas que emplean **Fine-tunning** tiene la flexibilidad de poder adaptar sus representaciones al permitir que todos los parametros sean optimizados en la tarea en particular. Además, estas arquitecturas permiten resolver multiples problemas de NLP utilizando una mínima cantidad de parametros específicos para la tarea.

## BETO: BERT en español

Al igual que con `word2vec`, entrenar un modelo de lenguaje requiere de una gran cantidad de datos sumado a un poder de computo interesante (cuando BERT fué publicado en 2018, tomó 4 días entrenar el modelo usando 16 TPUs. Si se hubiera entrenado en 8 GPUs hubiera tomado entre 40–70 días). Por este motivo, utilizaremos un modelo pre-entrenado para un cuerpo de texto en español. Este modelo, BETO, fué entrenado sobre un gran corpora de textos. Pueden encontrar más información sobre en [el sitio web del autor](https://github.com/dccuchile/beto).

### Tokenizers 

BERT utiliza su propio tokenizer que está basado en WordPiece. Este tokenizer tiene un vocabulario de 30.000 tokens donde cada secuencia comienza con un token especial [CLS]. Recuerden que los tokenizers dependen del modelo con el que estamos trabajando:

In [88]:
import transformers

tokenizer = transformers.BertTokenizerFast.from_pretrained('dccuchile/bert-base-spanish-wwm-uncased', 
                                                           do_lower_case=True)

loading file https://huggingface.co/dccuchile/bert-base-spanish-wwm-uncased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/eebf656e2fb33420d0d3f12a0650df76137cfd2251e04587d7d926fba30ab1b0.bfb98b35b81356261ec63a5ff66aa147928e2c8f4d09be77fc850582a1000498
loading file https://huggingface.co/dccuchile/bert-base-spanish-wwm-uncased/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/85478b69412001fdb7b4cb1f5e5c5e49df292e7de8a8a27c465348fd70e817e3.1fea6aa627ed25376d8778ace0885102803fe6651fb5638d1cea57cae8ccfa7f
loading file https://huggingface.co/dccuchile/bert-base-spanish-wwm-uncased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/dccuchile/bert-base-spanish-wwm-uncased/resolve/main/special_tokens_map.json from cache at /root/.cache/huggingface/transformers/78141ed1e8dcc5ff370950397ca0d1c5c9da478f54ec14544187d8a93eff1a26.f982506b52498d4adb4bd491f593dc92b2ef6be61bfdbe9d30f53f963f9f5b66
loading file

In [89]:
tokenizer.encode_plus(text="la casa estaba vacia")

{'input_ids': [4, 1032, 1635, 1594, 15912, 30956, 5], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

*Noten que el tokenizer depende del modelo que estamos utilizando*

## Crando un modelo de clasificación basado en BERT

Trataremos de resolver entonces el mismo problema de clasificación con el que veniamos trabajando: clasificar los tweets dependiendo del sector al que pertenecen.Recordemos que tenemos 7 categorias distintas:

In [7]:
tweets['SECTOR'].unique()

array(['RETAIL', 'TELCO', 'ALIMENTACION', 'AUTOMOCION', 'BANCA',
       'BEBIDAS', 'DEPORTES'], dtype=object)

Necesitaremos contar con el numero de categorias para nuestro clasificador:

In [8]:
num_labels=len(tweets['SECTOR'].unique())

Antes de hacer fine-tunning de nuestro modelo, tenemos que instanciar el modelo sobre el cual queremos aplicar esta técnica. Para ello instanciaremos el modelo base el cual no está entrenado en ninguna tarea en particular. De hecho, si habilitan las alertas en este notebook, verán que cuando se carga el modelo, la libreria HuggingFace les advierte sobre esto:

In [9]:
from transformers import BertForSequenceClassification
model = BertForSequenceClassification.from_pretrained('dccuchile/bert-base-spanish-wwm-uncased', 
                                                      num_labels=num_labels)

Downloading:   0%|          | 0.00/419M [00:00<?, ?B/s]

Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuc

Construiremos nuestro dataset sobre el que queremos entrenar el modelo. Recuerden que ya habíamos separado el set de datos en porciones para entrenar y para testear el modelo.

### Como entrenar modelos con Transformers

La librería transformers puede entrenar modelos tanto utilizando TensorFlow como PyTorch como backend. En nuestro caso utilizaremos PyTorch simplemente porque generaremos código un poco más compacto, pero pueden utilizar el backend con el que más cómodos se sientan:

### Set de datos

Para utilizar el objeto `Trainer` que provee `transformers`, necesitamos crear un objetivo de tipo `Dataset`. `PyTorch` implementa este objeto el modulo `torch.utils.data.Dataset`. Para simplificar la tarea, disponemos de una clase que hace todo el procesamiento de datos y generación de los conjuntos de datos utilizando dicho modulo. Pueden encontrar esta implementación en `Utils.ClassificationDataset`:

In [10]:
from Utils.TextDataset import ClassificationDataset

train_dataset = ClassificationDataset(examples=X_train, labels=y_train, tokenizer=tokenizer)
val_dataset = ClassificationDataset(examples=X_test, labels=y_test, tokenizer=tokenizer)

In [11]:
from transformers import Trainer, TrainingArguments

### Trainer

Especificamos los parametros con los que entrenaremos nuestro modelo:

In [12]:
training_args = TrainingArguments(
    output_dir='./results',          # Directorio de trabajo del Trainer
    num_train_epochs=3,              # Numero total de epochs sobre el que entrenaremos
    per_device_train_batch_size=16,  # Tamaño del batch de datos por cada dispositivo de entrenamiento
    per_device_eval_batch_size=64,   # Tamaño del batch de datos que usaremos para evaluación
    warmup_steps=500,                # Numero de pasos que se usaran para determinar la politica de Learning Rate
    weight_decay=0.01,               # Weight decay
    logging_dir='./logs',            # Directorio de logs
    logging_steps=100,
)

Instanciamos el Trainer

In [13]:
trainer = Trainer(
    model=model,                         # modelo sobre el que haremos fine tunning
    args=training_args,                  # parametros del entrenamiento
    train_dataset=train_dataset,         # set de datos de entrenamiento
    eval_dataset=val_dataset             # set de datos de evaluación
)

In [14]:
trainer.train()

***** Running training *****
  Num examples = 2521
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 474


Step,Training Loss
100,1.762200
200,0.512200
300,0.094200
400,0.046600




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=474, training_loss=0.5152663399901571, metrics={'train_runtime': 233.7099, 'train_samples_per_second': 32.361, 'train_steps_per_second': 2.028, 'total_flos': 310937225724000.0, 'train_loss': 0.5152663399901571, 'epoch': 3.0})

Verifiquemos la performance de nuestro modelo

In [15]:
predictions = trainer.predict(test_dataset=val_dataset).predictions

***** Running Prediction *****
  Num examples = 1242
  Batch size = 64


Para evaluar el modelo, primero deberemos obtener cual es la categoria que obtuvo la mayor probabilidad en la clasificación:

In [16]:
import numpy as np

predictions = np.argmax(predictions, axis=1)

Convertimos los IDs de las categorias a los labels correctos

In [17]:
all_labels = val_dataset.get_labels()
predictions_label = [all_labels[idx] for idx in predictions]

In [18]:
from sklearn.metrics import classification_report

print(classification_report(y_test, predictions_label))

              precision    recall  f1-score   support

ALIMENTACION       0.99      1.00      1.00       110
  AUTOMOCION       1.00      0.99      1.00       148
       BANCA       0.99      0.98      0.99       198
     BEBIDAS       1.00      0.98      0.99       223
    DEPORTES       0.96      1.00      0.98       216
      RETAIL       0.99      0.98      0.98       268
       TELCO       0.96      0.97      0.97        79

    accuracy                           0.99      1242
   macro avg       0.99      0.99      0.99      1242
weighted avg       0.99      0.99      0.99      1242



Persistimos el modelo

In [19]:
trainer.save_model('tweet_classification')
tokenizer.save_pretrained('tweet_classification')

Saving model checkpoint to tweet_classification
Configuration saved in tweet_classification/config.json
Model weights saved in tweet_classification/pytorch_model.bin
tokenizer config file saved in tweet_classification/tokenizer_config.json
Special tokens file saved in tweet_classification/special_tokens_map.json


('tweet_classification/tokenizer_config.json',
 'tweet_classification/special_tokens_map.json',
 'tweet_classification/vocab.txt',
 'tweet_classification/added_tokens.json',
 'tweet_classification/tokenizer.json')

In [ ]:
%%writefile tweet_classification.jsonnet

local transformer_model = "tweet_classification";
local transformer_dim = 768;

{
  "dataset_reader":{
    "type": "boolq",
    "token_indexers": {
      "tokens": {
        "type": "pretrained_transformer",
        "model_name": transformer_model,
      }
    },
    "tokenizer": {
      "type": "pretrained_transformer",
      "model_name": transformer_model,
    }
  },
  "train_data_path": "https://storage.googleapis.com/allennlp-public-data/BoolQ.zip!BoolQ/train.jsonl",
  "validation_data_path": "https://storage.googleapis.com/allennlp-public-data/BoolQ.zip!BoolQ/val.jsonl",
  "test_data_path": "https://storage.googleapis.com/allennlp-public-data/BoolQ.zip!BoolQ/test.jsonl",
  "model": {
    "type": "basic_classifier",
    "text_field_embedder": {
      "token_embedders": {
        "tokens": {
          "type": "pretrained_transformer",
          "model_name": transformer_model,
        }
      }
    },
    "seq2vec_encoder": {
       "type": "bert_pooler",
       "pretrained_model": transformer_model,
       "dropout": 0.1,
    },
    "namespace": "tags",
    "num_labels": 7,
  },
  "data_loader": {
    "batch_sampler": {
      "type": "bucket",
      "sorting_keys": ["tokens"],
      "batch_size" : 2
    }
  },
  "trainer": {
    "num_epochs": 10,
    "validation_metric": "+accuracy",
    "learning_rate_scheduler": {
      "type": "slanted_triangular",
      "num_epochs": 10,
      "num_steps_per_epoch": 3088,
      "cut_frac": 0.06
    },
    "optimizer": {
      "type": "huggingface_adamw",
      "lr": 1e-5,
      "weight_decay": 0.1,
    },
    "num_gradient_accumulation_steps": 16,
  },
}

In [ ]:
predictor = 

In [ ]:
trained_model = trainer.model

In [ ]:
indexed_tokens = tokenizer.encode("Nunca mas compro en Mercadona!")
tokens_tensor = torch.tensor([indexed_tokens]).to("cuda:0")

In [26]:
tokenizer.tokenize("Nunca mas compro en Mercadona!")

[[CLS], nunca, mas, compro, en, mercado, ##na, !, [SEP]]

In [ ]:
preds = trained_model(tokens_tensor)

In [ ]:
score, indices = torch.max(preds.logits, 1)

In [20]:
!pip install allennlp
!pip install -U google-cloud-storage==1.40.0

     |████████████████████████████████| 738 kB 4.0 MB/s 
     |████████████████████████████████| 12.1 MB 34.7 MB/s 
     |████████████████████████████████| 259 kB 39.5 MB/s 
     |████████████████████████████████| 124 kB 53.1 MB/s 
     |████████████████████████████████| 1.7 MB 40.2 MB/s 
     |████████████████████████████████| 190 kB 46.6 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 290 kB 43.6 MB/s 
     |████████████████████████████████| 1.2 MB 24.6 MB/s 
     |████████████████████████████████| 22.3 MB 20 kB/s 
     |████████████████████████████████| 167 kB 47.7 MB/s 
     |████████████████████████████████| 131 kB 54.5 MB/s 
     |████████████████████████████████| 79 kB 8.9 MB/s 
     |████████████████████████████████| 8.1 MB 37.0 MB/s 
     |████████████████████████████████| 138 kB 37.2 MB/s 
     |████████████████

     |████████████████████████████████| 104 kB 4.3 MB/s 
     |████████████████████████████████| 75 kB 4.3 MB/s 
  Attempting uninstall: google-resumable-media
    Found existing installation: google-resumable-media 0.4.1
    Uninstalling google-resumable-media-0.4.1:
      Successfully uninstalled google-resumable-media-0.4.1
  Attempting uninstall: google-cloud-core
    Found existing installation: google-cloud-core 1.0.3
    Uninstalling google-cloud-core-1.0.3:
      Successfully uninstalled google-cloud-core-1.0.3
  Attempting uninstall: google-cloud-storage
    Found existing installation: google-cloud-storage 1.18.1
    Uninstalling google-cloud-storage-1.18.1:
      Successfully uninstalled google-cloud-storage-1.18.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-bigquery 1.21.0 requires google-resumable-media!=0.4.0,<0.5.0dev,>=0.3.1, bu

In [342]:
from typing import Dict, Iterable, List

from allennlp.common import Params
from allennlp.data import DatasetReader, Instance, TextFieldTensors
from allennlp.data.fields import Field, LabelField, TextField, ArrayField, TensorField, TransformerTextField
from allennlp.data.token_indexers import TokenIndexer, SingleIdTokenIndexer
from allennlp.data.tokenizers import Token, Tokenizer, WhitespaceTokenizer

In [45]:
from allennlp.data.vocabulary import PreTrainedTokenizer, Vocabulary

In [38]:
from allennlp.modules.token_embedders import PretrainedTransformerEmbedder

In [62]:
params = Params({"model_name": "tweet_classification"})
embedder = PretrainedTransformerEmbedder.from_params(params)

In [31]:
from allennlp.models import BasicClassifier, Model

In [34]:
from allennlp.data.tokenizers.pretrained_transformer_tokenizer import PretrainedTransformerTokenizer
from allennlp.data.token_indexers.pretrained_transformer_indexer import PretrainedTransformerIndexer
from allennlp.modules.seq2vec_encoders.bert_pooler import BertPooler

In [90]:
tokenizer = PretrainedTransformerTokenizer("tweet_classification")
indexer = PretrainedTransformerIndexer("tweet_classification")
encoder = BertPooler("tweet_classification")

In [169]:
values = indexer.tokens_to_indices(tokenizer.tokenize("la casa estaba vacia"), vocabulary=model.vocab)

In [343]:
values

{'mask': [True, True, True, True, True, True, True],
 'token_ids': [4, 1032, 1635, 1594, 15912, 30956, 5],
 'type_ids': [0, 0, 0, 0, 0, 0, 0]}

In [83]:
model = BasicClassifier(vocab=Vocabulary(), text_field_embedder=embedder, seq2vec_encoder=encoder, dropout=0.1, num_labels=7)

In [292]:
from allennlp.data import DatasetReader

In [333]:
from allennlp_models.classification.dataset_readers import boolq

In [351]:
class ClassificationTransformerReader(DatasetReader):
    def __init__(
        self,
        tokenizer: Tokenizer,
        token_indexers: Dict[str, TokenIndexer],
        vocab,
        max_tokens: int,
        **kwargs
    ):
        super().__init__(**kwargs)
        self.tokenizer = tokenizer or WhitespaceTokenizer()
        self.token_indexers = token_indexers or {"tokens": SingleIdTokenIndexer()}
        self.max_tokens = max_tokens
        self.vocab = vocab

    def text_to_instance(self, text: str, label: str = None) -> Instance:
        tokens = self.tokenizer.tokenize(text)
        transformer_input = indexer.tokens_to_indices(tokens, vocabulary=self.vocab)
        if self.max_tokens:
            tokens = tokens[: self.max_tokens]
        
        #text_field = TextField(tokens, self.token_indexers)
        #fields: Dict[str, Field] = {"tokens": text_field }
        inputs = TransformerTextField(input_ids=transformer_input["token_ids"],
                                      token_type_ids=transformer_input["type_ids"],
                                      attention_mask=transformer_input["mask"])
        fields: Dict[str, Field] = {"tokens": inputs }
        if label:
            fields["label"] = LabelField(label)
        instance = Instance(fields)
        instance.indexed = True
        return instance

In [352]:
from allennlp.interpret.saliency_interpreters import SimpleGradient
from allennlp.predictors import Predictor, TextClassifierPredictor

In [353]:
dataset_reader = ClassificationTransformerReader(tokenizer=tokenizer, token_indexers={"tokens": indexer}, vocab=model.vocab, max_tokens=400)

In [358]:
instance = dataset_reader.text_to_instance("la vaca esta afuera")

In [359]:
dataset = Batch([instance])

In [360]:
dataset.index_instances(model.vocab)

In [362]:
from allennlp.nn import util
model_input = util.move_to_device(dataset.as_tensor_dict(), cuda_device)

NameError: ignored

In [354]:
predictor = TextClassifierPredictor(model, dataset_reader)

In [355]:
interpreter = SimpleGradient(predictor)

In [356]:
inputs = {"sentence": "a very well-made, funny and entertaining picture."}

In [357]:
interpretation = interpreter.saliency_interpret_from_json(inputs)

TypeError: ignored

In [ ]:
def plot_saliency(loaded_model, pure_txt, text_class, pred_labels, text_sequence):
    text_class = 'Positive' if text_class==1 else 'Negative'
    pred_labels = 'Positive' if pred_labels==1 else 'Negative'

    input_tensors = [loaded_model.input, K.learning_phase()]
    model_input = loaded_model.layers[2].input # the input for convolution layer
    model_output = loaded_model.output[0][1]
    gradients = loaded_model.optimizer.get_gradients(model_output,model_input)
    compute_gradients = K.function(inputs=input_tensors, outputs=gradients)
    matrix = compute_gradients([text_sequence.reshape(1,30), text_class])[0][0]
    matrix = matrix[:len(pure_txt),:]

    matrix_magnify=np.zeros((matrix.shape[0]*10,matrix.shape[1]))
    for i in range(matrix.shape[0]):
        for j in range(10):
            matrix_magnify[i*10+j,:]=matrix[i,:]

    fig = plt.figure()
    ax = fig.add_subplot(111)
    plt.imshow(normalize_array(np.absolute(matrix_magnify)), interpolation='nearest', cmap=plt.cm.Blues)
    plt.yticks(np.arange(5, matrix.shape[0]*10, 10), pure_txt, weight='bold',fontsize=24)
    plt.xticks(np.arange(0, matrix.shape[1], 50), weight='bold',fontsize=24)
    plt.title('True Label: "{}" Predicted Label: "{}"'.format(text_class,pred_labels), weight='bold',fontsize=24)
    plt.colorbar()
    plt.show()